In [1]:
import numpy as np
import pandas as pd

In [2]:
# hs, bec 파일 불러오기
hs_bec = pd.read_excel('HS2012-17-BEC5 -- 08 Nov 2018.xlsx', sheet_name='HS17BEC5')
hs_bec = hs_bec[['HS4','HS6','BEC5Code1', 'BEC5Processing', 'BEC5Specification']]

# hs code -> code 이름변경
hs_bec.rename(columns={'HS4':'HS4', 'HS6':'HS', 'BEC5Code1':'BEC', 'BEC5Processing':'BEC_Pro', 'BEC5Specification':'BEC_Spec'}, inplace=True)
hs_bec.head()
# hs_bec.size
# hs_bec.info()
hs_bec.to_csv('./hs_bec.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\project2\\BEC5Correlations\\통합 문서1.csv'

In [3]:
hs6_bec = pd.read_csv('C:\project2\Python\hs6_bec.csv')

# 6자리 맞춰주기 / str로 바꾼뒤 앞에 0붙여줌
hs6_bec['HS'] = hs6_bec['HS'].astype('str')
hs6_bec['HS'] = hs6_bec['HS'].str.zfill(width=6)

In [4]:
# korea exported 파일 불러오기
korea_exported_list = pd.read_table('C:\project2\Trade_map_List_of_products_exported_and_imported\Trade_Map_-_List_of_products_exported_by_Korea__Republic_of.txt', sep = '\t')

# code -> hs로 컬럼 이름 변경
korea_exported_list.rename(columns={'Code':'HS'}, inplace=True)
korea_exported_list.head()

,HS,Product label,Exported value in 2012,Exported value in 2013,Exported value in 2014,Exported value in 2015,Exported value in 2016,Exported value in 2017,Exported value in 2018,Exported value in 2019,Exported value in 2020,Exported value in 2021,Unnamed: 12
0,TOTAL,All products,547854448,559648708,573091134,526900733,495465606,573716618,605169190,542333337,512788606,644438622,NaN
1,854232,Electronic integrated circuits as memories,16611973,21964136,28849532,28856912,31612095,60506974,83048246,53163163,52359154,69177215,NaN
2,854231,Electronic integrated circuits as processors a...,21228826,20702672,18514980,19887934,17066083,21362697,22100868,20681846,23516687,30406085,NaN
3,271019,"Medium oils and preparations, of petroleum or ...",43536414,39290957,37799193,22784332,18938588,25256941,34171305,29751793,17844010,26178125,NaN
4,870323,Motor cars and other motor vehicles principall...,27767857,27237499,27337065,24060624,22007933,19133730,19612091,19701460,15352522,16416185,NaN


In [5]:
#   1) code 열에 total 제거
# 제거
korea_exported_list = korea_exported_list.drop([korea_exported_list.index[0]])

#   2) india 데이터 프레임에 hs 코드 데이터 타입 형변환
# korea_exported_list['HS'] = pd.to_numeric(korea_exported_list['HS'])
korea_exported_list['HS'] = korea_exported_list['HS'].astype('str')


#   2) merge를 사용해 hs코드 기준으로 데이터 프레임 병합
korea_exported_list = pd.merge(korea_exported_list, hs6_bec, how='left', on='HS')

#   2-1) hs 코드 자리수 맞춰주기 -> 5자리인경우 앞에 0붙여주기
korea_exported_list['HS'] = korea_exported_list['HS'].astype('str')
#       HS코드 6자리로 맞춰주기, 없는경우 left에 0으로 붙여줌
# korea_exported_list['HS'].str.pad(width=6, side='left', fillchar='0')
korea_exported_list['HS'] = korea_exported_list['HS'].str.zfill(width=6)

#   2-2 hs코드 잘라서 hs4에 넣어주기
korea_exported_list['HS4'] = korea_exported_list['HS'].str.slice(0, 4)


korea_exported_list.head()

,HS,Product label,Exported value in 2012,Exported value in 2013,Exported value in 2014,Exported value in 2015,Exported value in 2016,Exported value in 2017,Exported value in 2018,Exported value in 2019,Exported value in 2020,Exported value in 2021,Unnamed: 12,Unnamed: 0,HS4,BEC
0,854232,Electronic integrated circuits as memories,16611973,21964136,28849532,28856912,31612095,60506974,83048246,53163163,52359154,69177215,NaN,4858.0,8542,611220.0
1,854231,Electronic integrated circuits as processors a...,21228826,20702672,18514980,19887934,17066083,21362697,22100868,20681846,23516687,30406085,NaN,4857.0,8542,611220.0
2,271019,"Medium oils and preparations, of petroleum or ...",43536414,39290957,37799193,22784332,18938588,25256941,34171305,29751793,17844010,26178125,NaN,1067.0,2710,213101.0
3,870323,Motor cars and other motor vehicles principall...,27767857,27237499,27337065,24060624,22007933,19133730,19612091,19701460,15352522,16416185,NaN,4926.0,8703,513202.0
4,851770,"Parts of telephone sets, telephones for cellul...",7212942,10745179,13150064,17612803,14750059,6546331,6848687,12163895,12840863,15965206,NaN,4732.0,8517,611220.0


In [6]:
no_exported_id = korea_exported_list[
                    (korea_exported_list['Exported value in 2012'] == 0)
                    & (korea_exported_list['Exported value in 2013'] == 0)
                    & (korea_exported_list['Exported value in 2014'] == 0)
                    & (korea_exported_list['Exported value in 2016'] == 0)
                    & (korea_exported_list['Exported value in 2017'] == 0)
                    & (korea_exported_list['Exported value in 2018'] == 0)
                    & (korea_exported_list['Exported value in 2019'] == 0)
                    & (korea_exported_list['Exported value in 2020'] == 0)
                    & (korea_exported_list['Exported value in 2021'] == 0)].index

# 거래품목 없는 인덱스 제거
korea_exported_list = korea_exported_list.drop(no_exported_id)
korea_exported_list.isnull().sum()

HS                           0
Product label                0
Exported value in 2012       0
Exported value in 2013       0
Exported value in 2014       0
Exported value in 2015       0
Exported value in 2016       0
Exported value in 2017       0
Exported value in 2018       0
Exported value in 2019       0
Exported value in 2020       0
Exported value in 2021       0
Unnamed: 12               5182
Unnamed: 0                   1
HS4                          0
BEC                          3
dtype: int64

In [8]:
hs4_bec = hs6_bec[['HS4','BEC']]

# hs4, bec코드 정리
hs4_bec.dropna(axis=0, inplace= True)

hs4_bec['HS4'] = hs4_bec['HS4'].astype('int')
hs4_bec['BEC'] = hs4_bec['BEC'].astype('float')

C:\Users\user\AppData\Local\Temp\ipykernel_11788\3824369039.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs4_bec.dropna(axis=0, inplace= True)
C:\Users\user\AppData\Local\Temp\ipykernel_11788\3824369039.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs4_bec['HS4'] = hs4_bec['HS4'].astype('int')
C:\Users\user\AppData\Local\Temp\ipykernel_11788\3824369039.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [9]:
#### HS4 코드로 BEC NaN값 채우기

# 순수한 hs4코드를 모은 데이터베이스 만들기

# hs4_bec에서 hs4코드만 추출
hs4 = hs4_bec['HS4']

# hs4와 bec 값이 중복되는 행 제거
hs4.drop_duplicates(inplace=True)

# hs4 데이터프레임으로 형변환
hs4 = hs4.to_frame()

# 인덱스 초기화
hs4.reset_index(inplace=True)
hs4.count()

C:\Users\user\AppData\Local\Temp\ipykernel_11788\2588735676.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs4.drop_duplicates(inplace=True)


index    1222
HS4      1222
dtype: int64

In [10]:
# hs4에 대하여 bec 코드가 없는 행
# bec 값에 0
hs4_bec.loc[3513,'HS4'] = 0
hs4_bec.loc[3542,'HS4'] = 0

C:\Users\user\AppData\Local\Temp\ipykernel_11788\854863975.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs4_bec.loc[3513,'HS4'] = 0


In [12]:
# 사전가져오기
import pickle
# 사전 불러오기
with open('C:\project2\Python\hs4_bec.pkl', 'rb') as f:
    hs4_dict = pickle.load(f)

In [13]:
add_BEC = korea_exported_list[korea_exported_list['BEC'].isnull()]

# BEC 코드를 새로 부여하기 위해 기존 BEC 행 제거
add_BEC.drop(['BEC'], axis=1, inplace=True)

add_BEC.head()

C:\Users\user\AppData\Local\Temp\ipykernel_11788\868173811.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add_BEC.drop(['BEC'], axis=1, inplace=True)


,HS,Product label,Exported value in 2012,Exported value in 2013,Exported value in 2014,Exported value in 2015,Exported value in 2016,Exported value in 2017,Exported value in 2018,Exported value in 2019,Exported value in 2020,Exported value in 2021,Unnamed: 12,Unnamed: 0,HS4
1014,999999,Commodities not elsewhere specified,67648,64113,57712,56571,58393,49796,49332,46840,37392,39332,NaN,NaN,9999
1190,711890,Coin of legal tender,0,13875,16361,3497,8224,67050,52337,60841,78750,27431,NaN,3542.0,7118
4653,710820,Monetary gold,0,0,0,0,0,2,2,0,0,1,NaN,3513.0,7108


In [14]:
# 행 늘리기 위한 hs4 count 
hs4_bec_over = hs4_bec.drop_duplicates()
hs4_count = hs4_bec_over.groupby('HS4').count()

In [15]:
add_BEC_index = (add_BEC.index).to_list()
hs4_count_index = (hs4_count.index).to_list()

### 행 늘리기 - how
# 1. 늘려야하는 행들 확인
# 2. n개로 행 늘려주기
# 문제!!! 마지막 행이 항상 오류뜸 복사 붙여넣기 해주면 작동함
for i in add_BEC_index:
    com_key = int(add_BEC.loc[i, 'HS4'])
    for j in hs4_count_index:
        count = int(hs4_count.loc[j])       # count개수만큼 행 추가
        hs4Temp = int(j)
        # print(hs4Temp)
        # print(count, hs4Temp, com_key)
        if (count > 1) & (com_key == hs4Temp):  # 추가해야할 행이 2개이상이고, com_key와 hs4Tmep이 같을 경우
            for m in range(count - 1):  # count 개수만큼 행 추가
            #     # add_BEC = pd.concat([add_BEC, add_BEC.loc[[i]]], ignore_index = True)
                add_BEC = pd.concat([add_BEC, add_BEC.loc[[i]]], ignore_index = True)


In [16]:
# 위에서 행을 늘려주었기 때문에 인덱스 에러
# 인덱스 초기화
add_BEC.reset_index(inplace=True)

# 금액 나누기
for i in range(len(add_BEC)):
    com_key = int(add_BEC.loc[i, 'HS4'])
    for j in hs4_count_index:
        count = int(hs4_count.loc[j])       # count개수만큼 행 추가
        hs4Temp = int(j)
        if (count > 1) & (com_key == hs4Temp):
            for m in range(count):      # count 개수만큼 금액 나누기            
                add_BEC.loc[i,'Exported value in 2012'] = (add_BEC.loc[i, 'Exported value in 2012'] / count)
                add_BEC.loc[i,'Exported value in 2013'] = (add_BEC.loc[i, 'Exported value in 2013'] / count)
                add_BEC.loc[i,'Exported value in 2014'] = (add_BEC.loc[i, 'Exported value in 2014'] / count)
                add_BEC.loc[i,'Exported value in 2015'] = (add_BEC.loc[i, 'Exported value in 2015'] / count)
                add_BEC.loc[i,'Exported value in 2016'] = (add_BEC.loc[i, 'Exported value in 2016'] / count)
                add_BEC.loc[i,'Exported value in 2017'] = (add_BEC.loc[i, 'Exported value in 2017'] / count)
                add_BEC.loc[i,'Exported value in 2018'] = (add_BEC.loc[i, 'Exported value in 2018'] / count)
                add_BEC.loc[i,'Exported value in 2019'] = (add_BEC.loc[i, 'Exported value in 2019'] / count)
                add_BEC.loc[i,'Exported value in 2020'] = (add_BEC.loc[i, 'Exported value in 2020'] / count)
                add_BEC.loc[i,'Exported value in 2021'] = (add_BEC.loc[i, 'Exported value in 2021'] / count)
          

In [18]:
# HS4 코드 기준으로 SORT 
# 새로운 데이터프레임 add_BEC_sort에 넣기
add_BEC_sort = add_BEC.sort_values('HS4')
# add_BEC_sort_index = (add_BEC_sort.index).to_list()
add_BEC_sort.drop(['index'], axis=1, inplace = True)
add_BEC_sort.reset_index(inplace = True)


In [ ]:
# HS4코드 기준으로 dict에서 bec코드 각각 넣어주기
x = 0

for i in range(len(add_BEC_sort)):
    com_key = int(add_BEC_sort.loc[i, 'HS4'])
    for key, value in hs4_dict.items(): 
        # print( com_key,key, len(value))
        if (key == com_key):
            try:
                if len(value) == 1:
                    x = 0
                    # add_BEC_new.loc[i, 'HS4'] = key
                    add_BEC_sort.loc[i, 'BEC'] = value[x]
                    # print(add_BEC_sort.loc[i, 'BEC'])
                else:
                    add_BEC_sort.loc[i, 'BEC'] = value[x]
                    # print(i,key ,com_key, value[x], add_BEC_sort.loc[i, 'BEC'])
                    x = x + 1
                    if x == len(value):
                        x = 0
            except: IndexError

In [19]:
#### 원 데이터(bec 값 있는 것)와 add데이터 concat

#   1) ROW 데이터에서 BEC 코드 있는 데이터
korea_exported_list_bec = (korea_exported_list[korea_exported_list['BEC'].notnull()])

#   2) 모든 데이터들 넣어줄 데이터 프레임 생성
korea_exported = pd.DataFrame

#   3) add_BEC_sort와 korea_exported_list_bec concat
korea_exported = pd.concat([korea_exported_list_bec, add_BEC_sort])

In [21]:
# 모두 합친 df에서 BEC 값이 없는 데이터 확인
korea_exported[korea_exported['BEC'].isnull()]

,HS,Product label,Exported value in 2012,Exported value in 2013,Exported value in 2014,Exported value in 2015,Exported value in 2016,Exported value in 2017,Exported value in 2018,Exported value in 2019,Exported value in 2020,Exported value in 2021,Unnamed: 12,Unnamed: 0,HS4,BEC,index
0,710820,Monetary gold,0,0,0,0,0,2,2,0,0,1,NaN,3513.0,7108,NaN,2.0
1,711890,Coin of legal tender,0,13875,16361,3497,8224,67050,52337,60841,78750,27431,NaN,3542.0,7118,NaN,1.0
2,999999,Commodities not elsewhere specified,67648,64113,57712,56571,58393,49796,49332,46840,37392,39332,NaN,NaN,9999,NaN,0.0
